In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np
import re
import pickle
import modules.utils as Utils
from modules.regex_patterns import reviews_paths
from collections import Counter
from underthesea import word_tokenize
from sklearn.utils import shuffle
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import RandomOverSampler

### 20. Load file lên và shuffle nó

In [3]:
data = pd.read_csv("./data/standard_data.csv")

In [4]:
data = shuffle(data)

In [5]:
data

,raw_comment,standard_comment,label
17678,Đẹp hơn so vơi giá. Ưng,so vơi giá ưng,1
90738,Đẹp suất sắc dfbvvvbbbb.mua lần thứ ba rồi anz...,suất sắc lần thứ ba,1
187800,Shop làm ăn cẩu thả wa...gsjkdjsjbshdkksvsg xg...,ăn cẩu_thả làm_ăn,1
31207,Sản phẩm phù hợp với tầm giá.Mình thích mặc fo...,tầm giá thích rộng chọn tăng phù_hợp,1
143457,Cầ nm chắc tay không biết có bị phai màu hay k...,chắc tay phai hay,1
...,...,...,...
153184,Kẹp màu tím rất đẹp và dễ thương shop giao hàn...,kẹp tím nhanh tặng năm dễ_thương rất_đẹp,1
115295,Ok ạ nhưng bề rộng hơi nhỏ,bề_rộng hơi_nhỏ,0
83567,"Áo đẹp, vải mềm và mát, shop tư vấn chính xác ...",vải mềm mát tư_vấn chính_xác nhanh khác mang g...,1
25088,Xịn cực luôn,xịn cực,1


### 21. Kiểm tra số mẫu giữa các nhóm

In [6]:
no_samples_each_group = data[['label', 'standard_comment']].groupby(['label']).count()

In [7]:
no_samples_each_group

,standard_comment
label,
-1,3456
0,11842
1,203980


> **Nhận xét**:
> * Số lượng mẫu giữa các group không đều.
>
> $\Rightarrow$ thực hiện oversampling

### 22. Tạo tập `final_data`

In [8]:
no_samples_each_group = no_samples_each_group['standard_comment'].to_list()

In [9]:
negative = data.label[data.label.eq(-1)].sample(no_samples_each_group[0]).index
normal = data.label[data.label.eq(0)].sample(no_samples_each_group[1]).index
positive = data.label[data.label.eq(1)].sample(no_samples_each_group[1] + no_samples_each_group[0]*2).index

final_data = data.loc[negative.union(normal).union(positive)]

In [10]:
final_data = shuffle(final_data)

In [11]:
final_data

,raw_comment,standard_comment,label
208014,Dép mang êm chân mẫu sang hàng chất lượng,dép mang êm chân mẫu sang,1
62994,"Nón đẹp lắm ạ. Chất vải dày dặn, đường may đẹp...",nón vải dày_dặn chắc_chắn mặt thay_đổi tiện_lợ...,1
194727,"Hơi mỏng tí, nhưng giao đúng mẫu, dthg cực é. ...",tí đúng mẫu cực é hộ hơi_mỏng giao_đúng,0
164937,"Kính rất đẹp, shop tư vấn nhiệt tình, trả lời ...",kính tư_vấn nhiệt_tình trả_lời tin nhắn nhanh ...,1
162411,"Tính Tổng mét khối sdung\n=sumif( vùng dkien,...",tính tổng mét_khối vùng vùng tính tổng nhớ bấm...,1
...,...,...,...
180170,Sản phẩm giống hình. Chất liệu vải mềm.Giao hà...,liệu vải mềm nhanh ổn đóng_gói giống_hình,0
103094,Ví bỏ thẻ vào thôi cũng hơi kích 1 chút nhưng ...,ví bỏ thẻ hơi kích chút nỗi,1
74448,Áo mặc khá ổn. Giao hàng nhanh đúng mẫu. Điểm ...,nhanh đúng mẫu điểm trừ túi bé túi ngực giao_h...,0
11376,Sản phẩm hơi mỏng nhưng mà vs giá sale 1000d v...,giá tốt hơi_mỏng sale,0


In [12]:
final_data = final_data.reset_index(drop=True)

### 23. Thực hiện TF-IDF với `ngram_range=(1, 4)`

In [13]:
tf_idf = TfidfVectorizer(ngram_range=(1, 4))
tf_idf.fit(final_data['standard_comment'])

TfidfVectorizer(ngram_range=(1, 4))

### 24. Tạo tập input `X` và output `y`

In [14]:
X = tf_idf.transform(final_data['standard_comment'])
y = final_data['label']

### 25. Thực hiện over-sampling

In [15]:
ros = RandomOverSampler()
X_ros, y_ros = ros.fit_resample(X, y)

In [16]:
Counter(y_ros)

Counter({1: 18754, 0: 18754, -1: 18754})

### 26. Train test split

In [17]:
X_train, X_test, y_train, y_test = train_test_split(X_ros, y_ros, test_size=0.2, random_state=100, shuffle=True)

### 27. Lưu lại `final_data`, `tf_idf`, `X_train`, `X_test`, `y_train`, `y_test`

In [18]:
final_data.to_csv("./data/final_data.csv", index=False)

In [19]:
pickle.dump(tf_idf, open("./data/tf_idf.pickle", "wb"))
pickle.dump(X_train, open("./data/X_train.pickle", "wb"))
pickle.dump(X_test, open("./data/X_test.pickle", "wb"))
pickle.dump(y_train, open("./data/y_train.pickle", "wb"))
pickle.dump(y_test, open("./data/y_test.pickle", "wb"))